

## Прогнозирование стоимости автомобиля по характеристикам


In [291]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import pylab

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [292]:
#показывать dataframe без ограничения количества столбцов и 100 строк по умолчанию
pd.options.display.max_rows = 100
pd.options.display.max_columns = None

print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [293]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [294]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [295]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [296]:
#функция для первичного анализа категориальных столбцов
def plot_str(df,col):

    print('Распределение для столбца (не числовой):', col)
    fig,ax=plt.subplots(figsize=(10,5))
    sns.countplot(x=df.loc[:,col], ax=ax)
    plt.show()
#поиск пустых  Nan значений в символьном  столбце, расчет процента потерянных значений
    n=100-(df[col].count()/df.shape[0]*100)
    print('уникальных значений ', len(df[col].dropna().unique()))
    print ('пустых значений,%', round(n,2))
    #df.default.unique()

# Setup

In [297]:
# VERSION    = 16
# DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключение к ноутбуку внешнего датасета
# DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [298]:
# !ls '../input'

загрузка данных

In [299]:
train = pd.read_csv('../input/parsing-all-moscow-auto-ru-09-09-2020/all_auto_ru_09_09_2020.csv') # датасет для обучения модели
# bigtrain = pd.read_csv('../input/data-add/data1.csv')
mtrain=train.copy()

#train_add = pd.read_csv('../input/data-add/data1.csv')
test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
mtest=test.copy()

mtest.rename(columns={'model_name': 'model'}, inplace=True)
sample_submission = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

In [300]:
train.info()

In [301]:
train.head(5)

In [302]:
test.info()

Что может повлиять на стоимость

- 'bodyType' - тип кузова автомобиля, 
- 'brand', - производитель 
- 'fuelType' - тип топлива
- 'modelDate' - дата модели
- 'numberOfDoors' - количество дверей 
- 'productionDate' - дата производства
- 'vehicleTransmission' - вид трансмиссии 
- 'engineDisplacement' - объем двигателя 
- 'enginePower'- мощность двигателя
- 'mileage'- пробег 
- 'Привод'- привод 
- 'Руль' - руль
- 'ПТС' - наличеие ПТС 
- 'model' - модель

Числовые признаки : 'modelDate', 'productionDate',  'engineDisplacement', 'enginePower', 'mileage'.
Остальные признаки категориальные. 
Поэтому будем проводить моделирование в 2 вариантах: 
1. CatBoost, поскольку для ее использования не требуется делать hot encoding, подбор гиперпараметров по сетке.
2. Построение разрженной матрицы с помощью hot ecoding и тестирование различных ML библиотек.

Выбор модели с наилучшим результатом


**1. Исследуем признак - *'vehicleTransmission'***


In [303]:
mtest['vehicleTransmission'].value_counts()
plot_str(mtest,'vehicleTransmission')
plot_str(mtrain,'vehicleTransmission')

In [304]:
mtest.dropna(subset=['vehicleTransmission'], inplace=True)
#переименование данных  train и test

mtest.loc[mtest['vehicleTransmission'].str.contains("робот"),'vehicleTransmission'] =  "ROBOT"
mtest.loc[mtest['vehicleTransmission'].str.contains("мех"),'vehicleTransmission'] =  "MECHANICAL"
mtest.loc[mtest['vehicleTransmission'].str.contains("автомат"),'vehicleTransmission'] =  "AUTOMATIC"
mtest.loc[mtest['vehicleTransmission'].str.contains("вариат"),'vehicleTransmission'] =  "VARIATOR"
mtest.vehicleTransmission.value_counts()

2. **Исследуем признак *'fuelType'***

In [305]:
mtest['fuelType'].value_counts()

In [306]:
mtrain['fuelType'].value_counts()

расхождение в одно значение: универсал

In [307]:
mtrain[mtrain['fuelType']=='универсал']

эту строку можно вообще удалить, поскольку большинство значений NaN

In [308]:
mtrain.drop(labels = [24624], axis = 0, inplace = True)
mtrain['fuelType'].value_counts()

In [309]:
plot_str(mtest,'fuelType')
plot_str(mtrain,'fuelType')
mtest.info()

**3. Признак *'numberOfDoors'*** 

Количество дверей в автомобиле. Спорно, но возможно будет влиять на стоимость авто, поскольку косвенно подтверждает модель.
проверим, улучшится ли оценка предсказания. Признак категориальный.


In [310]:
mtest[ 'numberOfDoors' ].value_counts()

In [311]:
mtrain[ 'numberOfDoors' ].value_counts()

Здесь есть какое-то странное авто БЕЗ ДВЕРЕЙ.

In [312]:
mtrain[mtrain[ 'numberOfDoors' ]==0]

Это действительно реальный объект, есть в обеих выборках.

In [313]:
plot_str(mtest, 'numberOfDoors')
plot_str(mtrain, 'numberOfDoors')

4. **Исследуем признак *'engineDisplacement' (объем двигателя)***: 

In [314]:
print('Распределение для столбца (не числовой):', ' engineDisplacement')
fig,ax=plt.subplots(figsize=(10,70))
sns.countplot(y=mtrain.loc[:,'engineDisplacement'], ax=ax)
plt.show()

Проверим, как это выглядит.

In [315]:
mtrain[ 'engineDisplacement' ].unique()

In [316]:
mtrain[ 'engineDisplacement' ].value_counts()

По опыту, объем двигателя играет важную роль при принятии решения о покупке, поэтому данные стоит очистить. Выделим стандартные числовые показатели объема двигателя, остальные запишем как ноль. Во-первых, их число незначительно по сравнению со стандартными данными, во-вторых, якобы нулевой объем двигателя столь же информативен для массового покупателя, как и непонятные обозначения. Логично предположить, что его влияние будет сравнимо с нулевым показателем.

In [317]:
import re
def func_engineDisplacement(x):
    x1 = re.findall(r'\d\.\d', x)
    # print(x)
    if not x1: return '0'
    else: return x1[0]
    

In [318]:

mtrain.fillna(value=0, inplace=True)
mtrain['engineDisplacement']=mtrain['engineDisplacement'].apply(lambda x: func_engineDisplacement(x))
mtrain['engineDisplacement'].value_counts()

In [319]:
mtest['engineDisplacement'].value_counts()

Здесь данные более чистые, достаточно преобразовать их в числовой вид, учитывая строки, где число отсутствует.

In [320]:
mtest.dropna(subset=['engineDisplacement'], inplace=True)

In [321]:
def func_engineDisplecement_test(x):
    x1 = str(x)[:-4]
    if not x1: return 0
    else: return x1

In [322]:
func_engineDisplecement_test(' LTR')

In [323]:

mtest[ 'engineDisplacement' ] = mtest[ 'engineDisplacement' ].apply(lambda x: func_engineDisplecement_test(x))

In [324]:
mtest.head(20)

Вообще-то, по опыту, объем двигателя имеет значение для оценки автомобиля, но здесь какое-то огромное количество типов описания этого показателя.

In [325]:
#оставляем только данные в обучающей и тестовой выборке, где цена информативна: больше нуля
mtrain=mtrain[mtrain['price']>0]
train=train[train['price']>0]


Определяем поля - кандидаты для включения в модель

In [326]:
#формируем предварительный список нужных столбцов 
mcolumns=['bodyType', 'brand', 'fuelType', 'modelDate',
       'numberOfDoors', 'productionDate',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'mileage', 'Привод', 'Руль',
       'ПТС', 'price', 
       'model']


In [327]:
#### Удаляем строки с незаполненными значениями важных признаков

mtrain.dropna(subset=['bodyType'], inplace=True)
mtrain.dropna(subset=['Привод'], inplace=True)

**5. 'bodyType'** 

Выяснено, что есть большое разнообразие типов кузовов, однако многие, представленные в обоих датасетах по смыслу повторяют друг друга, но описаны по-разному. Необходимо привести значения к общему виду.

In [328]:
#приводим тип кузова к общим значениям
def rename_body (m):
    m['bodyType']=m['bodyType'].apply(lambda x: str.lower(x))
    m['bodyType']=m['bodyType'].str.replace(' ','')
    m.loc[m['bodyType'].str.contains("внедорожник3"),'bodyType'] =  "Внедорожник3"
    m.loc[m['bodyType'].str.contains("внедорожник5"),'bodyType'] =  "Внедорожник5"
    m.loc[m['bodyType'].str.contains("седанlimousine"),'bodyType'] =  "СеданLong"
    m.loc[m['bodyType'].str.contains("седанlong"),'bodyType'] = "СLong"
    m.loc[m['bodyType'].str.contains("седан-хардтоп"),'bodyType'] = "хардтоп"
    m.loc[m['bodyType'].str.contains("седан"),'bodyType'] = "Седан"    
    m.loc[m['bodyType'].str.contains( "минивэн"),'bodyType'] = "Минивэн"
    m.loc[m['bodyType'].str.contains( "компактвэн"),'bodyType'] = "Минивэн"
    m.loc[m['bodyType'].str.contains("универсал5"),'bodyType']  =  "Универсал5"
    m.loc[m['bodyType'].str.contains("хэтчбек5"),'bodyType'] = "Хэтчбек5"
    m.loc[m['bodyType'].str.contains("хэтчбек4"),'bodyType'] = "Хэтчбек5"
    m.loc[m['bodyType'].str.contains("пикапдв"),'bodyType'] = "Пикап2"
    m.loc[m['bodyType'].str.contains("пикапод"),'bodyType'] = "Пикап1"
    m.loc[m['bodyType'].str.contains("пикаппол"),'bodyType'] = "Пикап1_5"
    m.loc[m['bodyType'].str.contains("фургон"),'bodyType'] = "Фургон"
    m.loc[m['bodyType'].str.contains("родстер"),'bodyType'] = "Родстер"
    m.loc[m['bodyType'].str.contains("тарга"),'bodyType'] = "Родстер"
    m.loc[m['bodyType'].str.contains("кабриолет"),'bodyType'] = "Кабриолет"
    m.loc[m['bodyType'].str.contains("хэтчбек3"),'bodyType'] = "Хэтчбек3"
    m.loc[m['bodyType'].str.contains("лифтбек"),'bodyType'] = "Лифтбек"
    m.loc[m['bodyType'].str.contains("купе "),'bodyType'] = "Купе"
    m.loc[m['bodyType'].str.contains("микровэн"),'bodyType'] = "Микровэн"
    





In [329]:
#применяем измеения к тренировочной и тестовой выборке.
rename_body (mtrain)
rename_body (mtest)

**6. 'brand'**

Цель работы - построение модели для получения наилучшего результата на выборке в kaggle, поэтому есть смысл ограничить обучающую выбрку только брендами, которые присутствуют в тестовой выборке.


In [330]:
#Определили список брендов. Только на этих брендах есть смысл обучать модель
brand_list=mtest.brand.unique()

In [331]:
brand_list

In [332]:
#Сократим обучающую выбрку только до брендов, которые есть в тестовой выборке.
mtrain['brand']=mtrain['brand'].apply(lambda x:str.upper(x))
mtrain['bodyType']=mtrain['bodyType'].str.replace(' ','')

mtrain=mtrain[mtrain['brand'].isin (brand_list)]
mtrain=mtrain[mtrain['price']>0]


**7. Числовые столбцы: посмотрим на распределение и выбросы**

Оставляем 4 числовых столбца: 'productionDate','mileage','enginePower','modelDate'

In [333]:

#посмотрим на чиcловые столбцы и прологарифмируем их (эксперимент)
cols_to_ln=['productionDate','mileage','enginePower','modelDate']
print ('До логарифмирования')
fig, axes = plt.subplots(1, 4, figsize=(20,5))

for col, i in zip(cols_to_ln, range(4)):   
    sns.histplot(mtrain[col], kde=False, ax=axes.flat[i])
    
plt.show()



In [334]:
print ('После логарифмирования')
fig, axes = plt.subplots(1, 4, figsize=(20,5))

a=np.log(mtrain[cols_to_ln])
for col, i in zip(cols_to_ln, range(4)):#,'mileage','productionDate'
    
    sns.histplot(a[col], kde=False, ax=axes.flat[i])
    
pylab.show()

In [335]:
mtrain['mileage']=np.log(mtrain['mileage'])
mtest['mileage']=np.log(mtest['mileage'])
mtrain['productionDate']=np.log(mtrain['productionDate'])
mtest['productionDate']=np.log(mtest['productionDate'])
#Для тестовой выборки почистим столбец и преобразуем его в число
mtest['enginePower']=mtest['enginePower'].apply(lambda x: int(x[:-3].strip()))
mtrain['enginePower']=np.log(mtrain['enginePower'])
mtest['enginePower']=np.log(mtest['enginePower'])
mtrain['modelDate']=np.log(mtrain['modelDate'])
mtest['modelDate']=np.log(mtest['modelDate'])
mtest=mtest.replace(-np.Inf, 0)
mtest=mtest.replace(np.NINF, 0)
mtest=mtest.replace(np.Inf, 0)
mtrain=mtrain.replace(-np.Inf, 0)
mtrain=mtrain.replace(np.NINF, 0)
mtrain=mtrain.replace(np.Inf, 0)

## Data Preprocessing

In [336]:
train.dropna(subset=['productionDate','mileage','Руль','Привод','enginePower',
                     'modelDate','fuelType','numberOfDoors','vehicleTransmission','model'], inplace=True)
train.dropna(subset=['price'], inplace=True)

mtrain.dropna(subset=['productionDate','mileage','Руль','Привод','enginePower',
                      'modelDate','fuelType','numberOfDoors','vehicleTransmission','model'], inplace=True)
mtrain.dropna(subset=['price'], inplace=True)

mtrain=mtrain[mtrain['price']>0]


In [337]:
columns = ['bodyType', 'brand', 'productionDate',  'mileage','Руль',
           'Привод','enginePower','modelDate','fuelType','numberOfDoors','vehicleTransmission','model','engineDisplacement']
df_train = mtrain[columns]
df_test = mtest[columns]

In [338]:
y = mtrain['price']

## Label Encoding

In [339]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
mdata = data.copy()

**8. 'Руль'**
Приведем данные к одинаковым значениям

In [340]:
data['Руль']=data['Руль'].apply(lambda x: 'LEFT' if x =='Левый' else 'RIGHT')

for colum in ['bodyType', 'brand', 'Руль','Привод','fuelType','numberOfDoors','vehicleTransmission','model']:
    data[colum] = data[colum].astype('category').cat.codes #'engineDisplacement',


**9. 'model'** 
Есть смысл сравнить списки моделей из обучающей и тестовой выборок, сравнить 2 множества, посмотерть пересечение. Исключений не очень много, признак оставим, модель сильно влияет на стоимость авто.

In [341]:
tr=mtrain.model.unique().tolist()
ts=mtest.model.unique().tolist()
result=list(set(ts) - set(tr))
sorted(result)

In [342]:
data.sample(5)

In [343]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [344]:
union_data = mtest.append(mtrain, sort=False).reset_index(drop=True) # объединяем

## Train Split

In [345]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим первую модель 
Эта модель будет предсказывать среднюю цену по отобранным ранее признакам.



In [346]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

# # Model 1 : CatBoost

У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     


## Fit

In [347]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [348]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

На обработанных и отобранных признаках получили ошибку в 14%. 

### Log Traget
Попробуем взять таргет в Log - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
Также в этой модели используем параметры, подобранные по сетке. 

In [349]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         depth = 10, l2_leaf_reg = 7, learning_rate = 0.1)
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [350]:
#Оптимизируем гиперпараметры
#model_random = CatBoostRegressor()
#grid = {'learning_rate': [0.1, 0.2,0.3],
#        'depth': [ 6,8,10,12],
#        'l2_leaf_reg': [3,5,  7,9]}

#search_result = model_random.grid_search(grid, X=X_train, y=y_train, verbose=3, plot=True)
#print(search_result['params'])

In [351]:
#search_result['params']
#grid_cv = model_selection.GridSearchCV(CatBoostRegressor(), parameters_grid, scoring = 'MAPE', cv = cv)
#grid_cv.fit(X_train, X_test)

In [352]:
#model_random.best_params_

In [353]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [354]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Однако когда делаем submit, ошибка выратстает до 12.56.

## Пробуем другие ML модели: 
1.Подготовка train: labelIncoding


In [355]:
mdata.info()
cat_cols = ['bodyType', 'brand', 'Руль','Привод','fuelType','numberOfDoors','vehicleTransmission','model']#
num_cals=['productionDate','mileage','enginePower', 'modelDate']

Построим матрицу корреляций по чилосвым столбцам

In [356]:
plt.rcParams['figure.figsize'] = (15,10)

matrix = np.triu(mdata[mdata['sample'] == 0].corr())
sns.heatmap(mdata.corr(), annot=False, mask=matrix, cmap= 'coolwarm')

Видим, что дата выпуска сильно коррелирует с датой модели. Однако, при удалении одного из признаков ощутимо снижаетсяя оценка. Оставляем оба признака и позже проверим, возможно, из 2 признаков можно сделать 1.
Также есть достаточно сильная обратная корелляция этих признаков с пробегом.

In [357]:
#Определяем dummy-переменные
dummies = pd.get_dummies(mdata[cat_cols])

In [358]:
mdata_dum = pd.concat([mdata, dummies], axis=1)

In [359]:
mdata_dum.sample(5)

In [360]:
mdata_d = mdata_dum.drop(columns=cat_cols, axis=1)

In [361]:
mdata_d.sample(5)

In [362]:
XX = mdata_d.query('sample == 1').drop(['sample'], axis=1)
XX_sub = mdata_d.query('sample == 0').drop(['sample'], axis=1)

# Splitting the data
XX_train, X_val, yy_train, y_val = train_test_split(XX, y,  test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [363]:
yy_train=np.log(yy_train)

In [364]:
XX_train=XX_train.astype(float)
y_train=y_train.astype(float)
XX_train=XX_train.replace(-np.Inf, 0)
XX_train=XX_train.replace(np.NINF, 0)
XX_train=XX_train.replace(np.Inf, 0)
y_train=y_train.replace(-np.Inf, 0)
y_train=y_train.replace(np.NINF, 0)
y_train=y_train.replace(np.Inf, 0)
yy_train=yy_train.replace(-np.Inf, 0)
yy_train=yy_train.replace(np.NINF, 0)
yy_train=yy_train.replace(np.Inf, 0)

### XGBRegressor. 
Гиперпараметры были подобраны опытным путем, поскольку подбор по сетке занимал очень много времени.

In [365]:

xg_reg = xgb.XGBRegressor ( colsample_bytree= 0.7, 
                            learning_rate= 0.03, 
                            max_depth= 12, 
                            min_child_weight = 4, 
                            n_estimators = 500, 
                            nthread = 4, 
                            enable_categorical = True,
                            subsample = 0.7)

#(objective='reg:squarederror', colsample_bytree=0.5,
#                          learning_rate=0.05, max_depth=12, alpha=1,
#                          n_estimators=1000)
xg_reg.fit(XX_train, yy_train)

In [ ]:
predict_xg_reg = xg_reg.predict(X_val)
predict_xg_reg_sub = np.exp(xg_reg.predict(XX_sub))
#np.exp(model.predict(X_sub))
#display(predict)
#display(y_test)
print(f"Точность модели по метрике MAPE: {(mape(y_val, np.exp(predict_xg_reg)))*100:0.2f}%")

Submit - ошибка 13.46%

### Случайный лес

In [367]:
#  Random Forest
rf = ExtraTreesRegressor(n_estimators=300, random_state=RANDOM_SEED, n_jobs=-1,
                         bootstrap=True, verbose=1)
rf.fit(XX_train, yy_train)

In [368]:
pred_rf = rf.predict(X_val)
pred_rf_sub=np.exp(rf.predict(XX_sub))

In [369]:
MAPE = mape(y_test, pred_rf)
print(f"Точность модели по метрике MAPE: {(mape(y_val, np.exp(pred_rf)))*100:0.2f}%")
#print(f'Mean Absolute Percentage Error: {MAPE}')

### GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor(loss ='ls',n_estimators = 900, max_depth=10)
gbr.fit (XX_train, yy_train)

pred_gbr = gbr.predict(X_val)
pred_gbr_sub= np.exp(gbr.predict(XX_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_val, np.exp(pred_gbr)))*100:0.2f}%")

In [ ]:
pred_gbr_sub

In [ ]:
#GradientBoosting на  датасете, подготовленном для catboost
gbr = GradientBoostingRegressor(loss ='ls', max_depth=10, n_estimators = 1100)
gbr.fit (X_train, y_train)

In [ ]:
pred_gbr_1 = gbr.predict(X_test)
pred_gbr1_sub= gbr.predict(X_sub)
print(f"Точность модели по метрике MAPE: {(mape(y_test, pred_gbr_1))*100:0.2f}%")

pred_stack = clf.predict(X_test)
pred_stack_sub=np.exp(clf.predict(X_sub))

#подбор гиперпараметров для GradientBoostingRegressor
# Loss function to be optimized
loss = ['ls', 'lad', 'huber']

# Number of trees used in the boosting process
n_estimators = [100, 500, 900, 1100]

# Maximum depth of each tree
max_depth = [5, 10, 15]

# Minimum number of samples per leaf
min_samples_leaf = [ 4, 6, 8]

# Minimum number of samples to split a node
min_samples_split = [4, 6, 10]

# Maximum number of features to consider for making splits
max_features = ['auto', 'sqrt', 'log2', None]

# Define the grid of hyperparameters to search
hyperparameter_grid = {'loss': loss,
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'max_features': max_features}

# Create the model to use for hyperparameter tuning
model = GradientBoostingRegressor(random_state = 42)

# Set up the random search with 4-fold cross validation
GScv = GridSearchCV(model, hyperparameter_grid ,cv = 2, n_jobs = -1, verbose=True)

# Fit on the training data
GScv.fit(X_train, y_train) 
#After performing the search, we can inspect the RandomizedSearchCV object to find the best model:


Как видим точность возросла до 15%, а что будет на ЛБ?

# Submission

In [ ]:
sample_submission['price'] = pred_gbr_sub
sample_submission.to_csv(f'submission.csv', index=False)
#sample_submission.head(10)
#sample_submission['price'] = pred_rf_sub#*1.11
#sample_submission.to_csv(f'submission.csv', index=False)

In [ ]:
sample_submission

В итоге получили **MAPE 12,177%** 

Есть небольшая разница в размере ошибки между test и submission, которая наименее заметна при использовании  GradientBoostingRegressor.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing